Para o relatório parcial na foi necessário ser feita a reprodução do método de identificação proposto no trabalho da Carolina para o modelo MP.

## Modelo MP:

\begin{equation}
\tilde{y}(n) = \sum^P_{p=1}\sum^M_{m=0} \tilde{b}_{mp} \tilde{x}(n-m) |\tilde{x}(n-m)|^{p-1} 
\end{equation}


In [1]:
# IMPORTAÇÃO DAS BIBLIOTECAS E DADOS

from scipy.io import *
from scipy import *
from numpy import *
from matplotlib.pyplot import *


mat = loadmat('data_LDMOS.mat')
in_ext = mat['in_extraction']
out_ext = mat['out_extraction']
in_val = mat['in_validation']
out_val = mat['out_validation']

\begin{equation}

    X_{LUT} = 
    \begin{bmatrix}
        1blocodeQcolunasdalinha1 & 2blocodeQcolunasdalinha1 & \dots & (M+1)-ésimoblocodeQcolunasdalinha1\\
         & \vdots & & \\
        1blocodeQcolunasdalinha(n-M) & 2blocodeQcolunasdalinha(n-M) & \dots & (M+1)-ésimoblocodeQcolunasdalinha(n-M)\\
    \end{bmatrix}  
    
    
\end{equation}



\begin{equation} 
    X_{LUT} = 
    \begin{bmatrix}
    \tilde{x}(1+M) f_0^{LUT}(|\tilde{x}(1+M)|) & \tilde{x}(M) f_1^{LUT}(|\tilde{x}(M)|) & \dots &  \tilde{x}(1) f_M^{LUT}(|\tilde{x}(1)|)\\
     & \vdots & & \\
    \tilde{x}(n) f_0^{LUT}(|\tilde{x}(n)|) & \tilde{x}(n-1) f_1^{LUT}(|\tilde{x}(n-1)|) & \dots &  \tilde{x}(n-M) f_M^{LUT}(|\tilde{x}(n-M)|)
    \end{bmatrix}
\end{equation}


\begin{equation}
    Valor_{(q-1)_{esimacoluna}} = \tilde{x}(n-m) \left[ 1 - \frac{[|\tilde{x}(n-m)| - e_{m(q-1)}]}{e_{m(q)} - e_{m(q-1)}} \right]
\end{equation}

\begin{equation}
    Valor_{(q)_{esimacoluna}} = \tilde{x}(n-m) \left[ \frac{[|\tilde{x}(n-m)| - e_{m(q-1)}]}{e_{m(q)} - e_{m(q-1)}} \right]
\end{equation}



\begin{equation}
    s = (X_{LUT}^* X_{LUT})^{-1} (X_{LUT}^* Y) =
    \begin{bmatrix}
    \tilde{s}_{01} \\
    \vdots \\
    \tilde{s}_{0Q} \\
    \vdots \\
    \tilde{s}_{M1} \\
    \vdots \\
    \tilde{s}_{MQ}
    \end{bmatrix}
\end{equation}


In [246]:
# DEFINIÇÃO DOS PARÂMETROS DO MODELO
P = 5
M = 2

# n é o parâmetro que define o valor de Q (número de entradas da LUT), sendo Q=2^n
n = 1

# DEFINIÇÃO DO VETOR DE ENTRADA DAS LUTs
e_lut = linspace(0,1,2**n)


# DEFINIÇÂO DAS MATRIZES CONTENDO OS VALORES DE x(n-m) e |x(n-m)|
def x_n_m(entrada, M, P):
    x_nm = empty((len(entrada),(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(0,M+1,1):
            x_nm[r,m] = entrada[r-m]
    return x_nm

x_nm  = x_n_m(in_ext,M,P)[M+1:-1,:]
abs_x_nm = abs(x_nm)


In [247]:
def xlut(e_lut, x_nm, abs_x_nm, M, n):
    Q = 2**n
    x_lut = zeros((len(x_nm),Q),dtype = "complex_")
    for r in range(len(x_nm)):
        for c in range(1,Q,2):
            if e_lut[c-1] <= abs_x_nm[r,0] <= e_lut[c]:
                x_lut[r,c] = x_nm[r,0] * ((abs_x_nm[r,0] - e_lut[c-1])/((e_lut[c] - e_lut[c-1])))
                x_lut[r,c-1] = x_nm[r,0] * (1 - ((abs_x_nm[r,0] - e_lut[c-1])/((e_lut[c] - e_lut[c-1]))))

    if M>=1:
        for m in range(1, M+1):

            x_lut_temp = zeros((len(x_nm),Q),dtype = "complex_")
            for r in range(len(x_nm)):
                for c in range(1,Q,2):
                    if e_lut[c-1] <= abs_x_nm[r,m] <= e_lut[c]:
                        x_lut_temp[r,c] = x_nm[r,m] * ((abs_x_nm[r,m] - e_lut[c-1])/((e_lut[c] - e_lut[c-1])))
                        x_lut_temp[r,c-1] = x_nm[r,m] * (1 - ((abs_x_nm[r,m] - e_lut[c-1])/((e_lut[c] - e_lut[c-1]))))

            x_lut = concatenate((x_lut,x_lut_temp), axis=1)                 
    return x_lut

x_lut = xlut(e_lut, x_nm, abs_x_nm, M, n)
x_lut

array([[0.02452544-0.21704066j, 0.01166367-0.10321899j,
        0.03309099-0.17495131j, 0.0099834 -0.05278202j,
        0.047343  -0.10958357j, 0.00761603-0.01762861j],
       [0.02254974-0.2380096j , 0.01475211-0.15570664j,
        0.02452544-0.21704066j, 0.01166367-0.10321899j,
        0.03309099-0.17495131j, 0.0099834 -0.05278202j],
       [0.0272328 -0.24503633j, 0.02150343-0.19348438j,
        0.02254974-0.2380096j , 0.01475211-0.15570664j,
        0.02452544-0.21704066j, 0.01166367-0.10321899j],
       ...,
       [0.02194483-0.19063959j, 0.00766863-0.06661903j,
        0.02432916-0.17144672j, 0.00697475-0.04915078j,
        0.03517417-0.15074186j, 0.00832811-0.03569083j],
       [0.02714023-0.20651418j, 0.01139862-0.08673385j,
        0.02194483-0.19063959j, 0.00766863-0.06661903j,
        0.02432916-0.17144672j, 0.00697475-0.04915078j],
       [0.03849435-0.21755455j, 0.01891811-0.10691751j,
        0.02714023-0.20651418j, 0.01139862-0.08673385j,
        0.02194483-0.19063959j,

In [248]:
x1 = asmatrix(x_lut).H
s_lut = asmatrix(x1@x_lut).I@asmatrix(x1@out_ext[M+1:-1,:])

lut = empty((int(len(s_lut)/(M+1)), 2*(M+1)))
rl = 0
for c in range(1,2*(M+1),2):
    for r in range(2**n):
        lut[r,c] = imag(s_lut[rl])
        lut[r,c-1] = real(s_lut[rl])
        rl+=1
lut

array([[ 1.11720139,  0.06746139, -0.03581887, -0.01435655, -0.01774943,
         0.00814816],
       [ 0.70252877, -0.08186121,  0.31861196, -0.02379433, -0.12042486,
        -0.00615358]])

In [249]:
def interpolacao(entrada,lut,M,n):
    Q = 2**n
    z = np.linspace(0,1,Q) 
    abs_in = np.abs(entrada)
    abs_in = abs_in[M+1:-1,:]
    inter = np.interp(abs_in,z,lut[:,0])
    for c in range(1,2*(M+1)):
        inter2 = np.interp(abs_in,z,lut[:,c])
        inter = np.append(inter,inter2,axis=1)
    
    real = np.zeros((len(inter),1))
    imag = np.zeros((len(inter),1))
    for c in range(0,2*(M+1),2):
        real = np.add(real,inter[:,c].reshape(len(inter),1))
        imag = np.add(imag,inter[:,c+1].reshape(len(inter),1))
    interpol = real + (imag * 1j)
    return interpol

inter = interpolacao(in_val,lut,M,n)
in_val_lut = in_val[M+1:-1,:]
out_val_lut = out_val[M+1:-1,:]
out_lut = inter*in_val_lut

In [250]:
def NMSE(previsao,validacao):
    erro = validacao - previsao
    erro = np.sum(np.absolute(erro)**2)
    val = np.sum(np.absolute(validacao)**2)
    res = 10*np.log10(erro/val)
    return res

NMSE_lut = NMSE(out_lut,out_val_lut)
print(NMSE_lut)

-31.38759849608664
